# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
from tld import get_tld

# Define directories

In [2]:
number = '20'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Open clean dataset

In [3]:
data_file = pd.read_csv(analyze_dir + 'csv_files/clean_files/clean_files_forum_' + number + '.csv')
data_file

,file_number,message_id
0,1000000,message #1622278
1,1000003,message #1622281
2,1000004,message #1622282
3,1000005,message #1622283
4,1000006,message #1622284
5,1000009,message #1622287
6,1000010,message #1622288
7,1000011,message #1622289
8,1000014,message #1622292
9,1000015,message #1622293


In [4]:
data_file.iloc[1][0]

1000003

# Get links from files

In [5]:
print ("Forum files: %s" %(number))
url_data = []
for file_num in range(0,len(data_file)):
    sys.stdout.write('\rExtracting file number: %i' %(file_num+1) + ' / %i' %(len(data_file)))
    sys.stdout.flush()

    file = dataset + str(data_file.iloc[file_num][0])
    f = open(file)
    f = f.read()

    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [data_file.iloc[file_num][0], msid, title]
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a'):
                    username = usr.text
                    if username == '':
                        username = 'Eclipse User'

                    for link in userlink.find_all('span', attrs={'class':'MsgBodyText'}):
                        for atag in link.find_all('a'):
                            ss = atag.text.replace(' ','')
                            parser = []
                            try:
                                link = re.search('^https?:\/\/.*[\r\n]*[^.\\,:;]', ss)
                                link = link.group()
                                domain = urlparse(link)
                                tld_dt = get_tld(link)
                                parser = [username, link, domain.netloc, tld_dt]
                                data.insert(len(data),parser)
                            except:
                                pass

        n = 3
        link_name = []
        for i in range (0,len(data)-3):
            if data[n][1] not in link_name:
                url_data.append([data[0], data[1], data[2], data[n][0], data[n][1], data[n][2], data[n][3]])
                link_name.append(data[n][1])
            else:
                pass
            n = n + 1
    
    except:
        pass

print ('\nFiles extraction are finished')

Forum files: 20
Extracting file number: 142598 / 142598
Files extraction are finished


In [6]:
url_data

[[1000000,
  'message #1622278',
  'Welcome to eclipse.camf',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [1000000,
  'message #1622278',
  'Welcome to eclipse.camf',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.camf',
  'www.eclipse.org',
  'org'],
 [1000004,
  'message #1622282',
  'Welcome to eclipse.che',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [1000004,
  'message #1622282',
  'Welcome to eclipse.che',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.che',
  'www.eclipse.org',
  'org'],
 [1000005,
  'message #1622283',
  'Welcome to eclipse.titan',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [1000005,
  'message #1622283',
  'Welcome to eclipse.titan',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.titan',
  'www.eclipse.org',
  'org'],
 [1000006,
  'message #1622284',
  'Welcome

In [7]:
with open (analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'topic', 'author', 'link', 'domain', 'top_level_domain']
    writers.writerow(header)
    for item in url_data:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
df_link = pd.read_csv(analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv')
df_link

,file_number,message_id,topic,author,link,domain,top_level_domain
0,1000000,message #1622278,Welcome to eclipse.camf,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
1,1000000,message #1622278,Welcome to eclipse.camf,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.camf,www.eclipse.org,org
2,1000004,message #1622282,Welcome to eclipse.che,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
3,1000004,message #1622282,Welcome to eclipse.che,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.che,www.eclipse.org,org
4,1000005,message #1622283,Welcome to eclipse.titan,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
5,1000005,message #1622283,Welcome to eclipse.titan,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.titan,www.eclipse.org,org
6,1000006,message #1622284,Welcome to eclipse.camf,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
7,1000006,message #1622284,Welcome to eclipse.camf,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.camf,www.eclipse.org,org
8,1000009,message #1622287,Welcome to eclipse.che,Eclipse Webmaster,http://www.eclipse.org/proposals/,www.eclipse.org,org
9,1000009,message #1622287,Welcome to eclipse.che,Eclipse Webmaster,http://www.eclipse.org/forums/eclipse.che,www.eclipse.org,org


# Remove links if the author is Eclipse Webmaster

In [9]:
cols = ['file_number','message_id','topic','author','link','domain','top_level_domain']
new_df_link = df_link[cols][df_link[cols]['author'] != 'Eclipse Webmaster']
new_df_link

,file_number,message_id,topic,author,link,domain,top_level_domain
14,1000014,message #1622292,[ETL] Standalone example - IRelativePathResolv...,Dimitris Kolovos,http://www.eclipse.org/downloads/download.php?...,www.eclipse.org,org
15,1000014,message #1622292,[ETL] Standalone example - IRelativePathResolv...,devi kumar,http://www.eclipse.org/epsilon/examples/index....,www.eclipse.org,org
16,1000014,message #1622292,[ETL] Standalone example - IRelativePathResolv...,Dimitris Kolovos,https://www.youtube.com/watch?v=6jRFLrEC0l4,www.youtube.com,com
17,1000014,message #1622292,[ETL] Standalone example - IRelativePathResolv...,Dimitris Kolovos,http://www.eclipse.org/epsilon/examples/index....,www.eclipse.org,org
36,1000031,message #1622310,Where can I find Eclipse Java IDE installation...,Eike Stepper,http://www.eclipse.org/downloads/index-develop...,www.eclipse.org,org
37,1000031,message #1622310,Where can I find Eclipse Java IDE installation...,Eike Stepper,http://www.esc-net.de,www.esc-net.de,de
38,1000031,message #1622310,Where can I find Eclipse Java IDE installation...,Eike Stepper,http://thegordian.blogspot.com,thegordian.blogspot.com,blogspot.com
39,1000031,message #1622310,Where can I find Eclipse Java IDE installation...,Eike Stepper,http://twitter.com/eikestepper,twitter.com,com
232,1000198,message #1622495,String problem with text replace,Alexandra Schladebeck,http://help.eclipse.org/luna/topic/org.eclipse...,help.eclipse.org,org
233,1000198,message #1622495,String problem with text replace,Alexandra Schladebeck,http://www.uni-regensburg.de/EDV/Misc/KeyBoard...,www.uni-regensburg.de,de


In [10]:
new_df_link.to_csv(analyze_dir + 'csv_files/links_list/cleaned/links_without_bot_authors_forum_' + number + '.csv', index=False)
print ("CSV file has been created")

CSV file has been created
